### Save Checkpoint (MNIST)

This notebook is used to train the best models found during training. \
They will be fully trained and the best saved as a checkpoint for use in the demo notebook.

In [ ]:
# Install dependencies
%pip install --upgrade pip
%pip install numpy
%pip install seaborn
%pip install matplotlib
%pip install pandas
%pip install scikit-learn
%pip install tensorflow[and-cuda]

In [ ]:
# Import dependencies
import os
import matplotlib.pyplot as plt # type: ignore
import matplotlib.gridspec as gridspec # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf # type: ignore
import seaborn as sns # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # type: ignore
from tensorflow.keras.datasets import mnist # type: ignore
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D # type: ignore
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay # type: ignore

In [ ]:
# Tensorflow version
print(f"Tensorflow: v{tf.__version__}")

# Check GPU availability
print(f"GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Set seed for reproducibility
SEED = 42
np.random.seed(SEED)

In [ ]:
# Load results csv files
csv_files = [
    f'../results/mnist/{file}'
    for file in os.listdir('../results/mnist/')
    if file.endswith('.csv') and file != 'all.csv'
]

# Combine and clean the results
all_results = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
all_results = all_results.drop(columns=['Unnamed: 0'])
all_results = all_results.round({ 'dropout_rate': 1 })

# Save the combined results to a csv file
all_results = all_results.sort_values('val_accuracy', ascending=False)
all_results.to_csv('../results/mnist/all.csv', index=False)

# Get the top 3 models from the combined results
top_models = all_results.head(3)
top_models

In [ ]:
# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape data to add a channel dimension, normalize pixel values
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Display some info and stats about the dataset
print(f'Training data shape: {X_train.shape}')
print(f'Test data shape: {X_test.shape}')
print(f'Number of classes: {y_train.shape[1]}')

In [ ]:
# Define the model
def create_custom_model(conv1, conv2, dense_units, dropout_rate, learning_rate):
    # Build the model architecture
    model = Sequential([
        Input(shape=(28, 28, 1)),
        Conv2D(conv1, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(conv2, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(10, activation='softmax')
    ])
    
    # Compile the model with the chosen learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Define the plotting function
def plot_training_history(model, history, hyperparameters, index):
    # Generate predictions for the test set
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Combined figure for accuracy, loss, and confusion matrix
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))

    # Model's accuracy
    axes[0].plot(history['accuracy'], label='Train Accuracy')
    axes[0].plot(history['val_accuracy'], label='Validation Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend(loc='lower right')
    axes[0].set_title('Training and Validation Accuracy')
    axes[0].grid(True)

    # Model's loss
    axes[1].plot(history['loss'], label='Train Loss')
    axes[1].plot(history['val_loss'], label='Validation Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend(loc='upper right')
    axes[1].set_title('Training and Validation Loss')
    axes[1].grid(True)

    # Model's classification report
    report = classification_report(y_true, y_pred_classes, output_dict=True)
    df = pd.DataFrame(report).transpose()
    sns.heatmap(df.iloc[:-1, :-3], annot=True, fmt=".2f", cmap='Blues', ax=axes[2])
    axes[2].set_title('Classification Report')
    axes[2].grid(False)

    # Model's Confusion matrix
    confusion_mtx = confusion_matrix(y_true, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mtx, display_labels=np.arange(10))
    disp.plot(cmap=plt.cm.Blues, ax=axes[3], colorbar=False)
    axes[3].set_title('Confusion Matrix')
    axes[3].grid(False)

    # Include the hyperparameters in the title
    title = f"Validation Accuracy: {float(hyperparameters['val_accuracy']):.5f}   Hyperparameters: Conv1: {int(hyperparameters['conv1'])}, Conv2: {int(hyperparameters['conv2'])}, Dense: {int(hyperparameters['dense_units'])}, Dropout: {float(hyperparameters['dropout_rate']):.1f}, Learning Rate: {float(hyperparameters['learning_rate']):.5f}, Batch Size: {int(hyperparameters['batch_size'])}"
    fig.suptitle(title)

    # Adjust layout and display the figure
    plt.tight_layout()

    # Save the figure
    os.makedirs('../results/mnist', exist_ok=True)
    plt.savefig(f'../results/mnist/best-{index}.png')

    # Display the figure
    plt.show()

In [ ]:
# Train the top models and save the results
for i, (_, row) in enumerate(top_models.iterrows(), 1):
    # Build the model with the hyperparameters
    model = create_custom_model(
        int(row['conv1']),
        int(row['conv2']),
        int(row['dense_units']),
        float(row['dropout_rate']),
        float(row['learning_rate'])
    )

    # Add callbacks for model training
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            '../results/mnist/best-model.keras',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ) if i == 1 else None
    ]

    # Train the model
    history = model.fit(
        X_train, y_train, 
        validation_data=(X_test, y_test),
        epochs=100,
        batch_size=int(row['batch_size']),
        callbacks=[cb for cb in callbacks if cb is not None],
    ).history

    # Plot the training history
    plot_training_history(model, history, row, i)